# R Aqr offsets calculations
Calculate radial offset and PA angle for R Aqr target acquisition.

In [11]:
# Set up autoloader
%load_ext autoreload
%autoreload 2

In [3]:
# Set up modules and stuff
import numpy as np
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord, Distance

## Date range for JWST observation
This is the date range for the scheduled JWST observation. For now the schedules time is unknown, and is fixed to the middle point in the observation window.

In [4]:
# JWST dates and times
JWST_TIME1 = '2024-10-30T18:19:00.000'    # Beginning of JWST observing window
JWST_TIME2 = '2024-12-12T00:19:55.000'    # Beginning of JWST observing window
JWST_TIME = '2024-11-06T18:19:00.000'     # Schediuled time JWST time

# Create Time-like objects with the JWST times
jwst_time1 = Time(JWST_TIME1, format='isot', scale='utc')
jwst_time2 = Time(JWST_TIME2, format='isot', scale='utc')
jwst_time = Time(JWST_TIME, format='isot', scale='utc')

## Reference coordinates for R Aqr and offset star
We provide two sets of coordinates for R Aqr (since the target had been observed twice), and the Gaia DR3 Coodinates of the offset star. The most recent set is the one actually used to calculate the updated coordinates at the time of the JWST observation. The earlier set of coordinates is used to test that the routine work properly (by using them as reference coordinates and checking that the result matches).

In [5]:
# R Aqr coordinates set 1
RA = '23:43:49.50120 hours'     # RA in sexagesimal format (ICRS)
DEC = '-15:17:04.804 degrees'   # DEC in sexagesimal format (ICRS) 
PMRA = 27.33                    # Proper motion in RA (mas/yr)
PMDEC = -30.37                  # Proper motion in DEC (mas/yr)
#PARALLAX = 2.5931              # Gaia DR3 Parallax (mas)
PARALLAX = 4.59                 # Adriantsaralaza et al. 2022 parallax (mas)
OBS_DATE1 = '2020-10-01'        # Observation date
EPOCH = 2000.0                  # Reference epoch
FRAME = 'icrs'                  # Coordinate frame

# Create Skycoord object with the given coordinates
r_aqr1 = SkyCoord(RA,
                  DEC,
                  pm_ra_cosdec=PMRA*u.mas/u.yr,
                  pm_dec=PMDEC*u.mas/u.yr,
                  distance=Distance(parallax=PARALLAX*u.mas),
                  equinox=Time(EPOCH, format='jyear'),
                  obstime=Time(OBS_DATE1, format='iso', scale='utc'),
                  frame=FRAME
                 )

In [6]:
# R Aqr coordinates set 2
RA = '23:43:49.50785 hours'     # RA in sexagesimal format (ICRS)
DEC = '-15:17:04.911 degrees'   # DEC in sexagesimal format (ICRS) 
PMRA = 27.33                    # Proper motion in RA (mas/yr)
PMDEC = -30.37                  # Proper motion in DEC (mas/yr)
#PARALLAX = 2.5931              # Gaia DR3 Parallax (mas)
PARALLAX = 4.59                 # Adriantsaralaza et al. 2022 parallax (mas)
OBS_DATE2 = '2024-04-07'        # Observation date
EPOCH = 2000.0                  # Reference epoch
FRAME = 'icrs'                  # Coordinate frame

# Create Skycoord object with the given coordinates
r_aqr2 = SkyCoord(RA,
                  DEC,
                  pm_ra_cosdec=PMRA*u.mas/u.yr,
                  pm_dec=PMDEC*u.mas/u.yr,
                  distance=Distance(parallax=PARALLAX*u.mas),
                  equinox=Time(EPOCH, format='jyear'),
                  obstime=Time(OBS_DATE2, format='iso', scale='utc'),
                  frame=FRAME
                 )

In [7]:
# Print total and PA of offset between the two sets of coordinates
tot_offset = r_aqr1.separation(r_aqr2).to(u.arcsec)
pa_offset = r_aqr1.position_angle(r_aqr2).to(u.deg)
print('--> Offset between R_Aqr 1 and R_Aqr 2 coordinates:')
print(f'--> Total offset:   {tot_offset:.6f}')
print(f'--> Position angle: {pa_offset:.4f}')

--> Offset between R_Aqr 1 and R_Aqr 2 coordinates:
--> Total offset:   0.143901 arcsec
--> Position angle: 138.0360 deg


In [8]:
# Gaia DR3 2419576423272148480 coordinates
RA = '355.94560363551375 degrees'    # RA in decimal degrees (ICRS)
DEC = '-15.274577751934695 degrees'  # DEC in decimal degrees (ICRS) 
PMRA = -2.708690975158548            # Proper motion in RA (mas/yr)
PMDEC = -4.0809858570655875          # Proper motion in DEC (mas/yr)
OBS_DATE = 2016.0                    # Observation date
EPOCH = 2016.0                       # Reference epoch
FRAME = 'icrs'                       # Coordinate frame

# Create Skycoord object with the given coordinates
acqt = SkyCoord(RA,
                DEC,
                pm_ra_cosdec=PMRA*u.mas/u.yr,
                pm_dec=PMDEC*u.mas/u.yr,
                equinox=Time(EPOCH, format='jyear'),
                obstime=Time(OBS_DATE, format='jyear', scale='tcb'),
                frame=FRAME
            )

In [9]:
# Print offset between R Aqr and acquisition star
# Based on their nominal coordinates (position 1)
offset1 = r_aqr1.separation(acqt).to(u.arcsec)
pa_offset1 = r_aqr1.position_angle(acqt).to(u.deg)
print('--> Offset between R Aqr1 and acquisition target:')
print()
print(f'--> Total Offset (with R Aqr 1):   {offset1:.6f}')
print(f'--> Position angle (with R Aqr 1): {pa_offset1:.4f}')
print()

# Print offset between R Aqr and Gaia DR3 2419576423272148480
# Based on their nominal coordinates (position 2)
offset2 = r_aqr2.separation(acqt).to(u.arcsec)
pa_offset2 = r_aqr2.position_angle(acqt).to(u.deg)
print(f'--> Total Offset (with R Aqr 2):   {offset2:.6f}')
print(f'--> Position angle (with R Aqr 2): {pa_offset2:.4f}')


--> Offset between R Aqr1 and acquisition target:

--> Total Offset (with R Aqr 1):   51.842649 arcsec
--> Position angle (with R Aqr 1): 314.4786 deg

--> Total Offset (with R Aqr 2):   51.986274 arcsec
--> Position angle (with R Aqr 2): 314.4884 deg


## Update coordinates for R Aqr and acquisition star
Update position of R Aqr and acquisition star to initial and final date of observing window. Use the apply_space_motion() method of the SkyCoord package. Calculate also position of R Aqr at the time of the second observation based on the position in the first observation, as a sanity check.

In [12]:
# Update latest coordinates of R Aqr to beggining of JWST observing window
r_aqr2_time1 = r_aqr2.apply_space_motion(new_obstime=jwst_time1)

# Update latest coordinates of R Aqr to the end of JWST observing window
r_aqr2_time2 = r_aqr2.apply_space_motion(new_obstime=jwst_time2)

# Update latest coordinates of R Aqr to the scheduled JWST time
r_aqr2_time = r_aqr2.apply_space_motion(new_obstime=jwst_time)

# Print offset between original and updated coordinates of R Aqr (JWST time)
offset_jwst = r_aqr2.separation(r_aqr2_time).to(u.mas)
pa_offset_jwst = r_aqr2.position_angle(r_aqr2_time).to(u.deg)
print('--> Offset between R Aqr 2 and R Aqr at JWST observing time:')
print(f'--> Total Offset:   {offset_jwst:.3f}')
print(f'--> Position angle: {pa_offset_jwst:.2f}')

--> Offset between R Aqr 2 and R Aqr at JWST observing time:
--> Total Offset:   23.911 mas
--> Position angle: 138.02 deg


In [13]:
# Calculate the offset between the two sets of coordinates
# with the first set updated to the time of the second set.
# This is just a sanity check to see if the offset is close to zero.
r_aqr1_to2 = r_aqr1.apply_space_motion(new_obstime=r_aqr2.obstime)
offset = r_aqr1_to2.separation(r_aqr2).to(u.mas)
print("--> Offset between the first and second set of R Aqr coordinates")
print("--> at the time of the second set of coordinates: ")
print(f'--> Total offset: {offset:.3f}')
print("--> Note: that the result should be as close as possible to zero <---")

--> Offset between the first and second set of R Aqr coordinates
--> at the time of the second set of coordinates: 
--> Total offset: 0.278 mas
--> Note: that the result should be as close as possible to zero <---


In [15]:
# Update acquisition target coordinates to beggining of JWST observing window
acqt_time1 = acqt.apply_space_motion(new_obstime=jwst_time1)

# Update acquisition target coordinates to end of JWST observing window
acqt_time2 = acqt.apply_space_motion(new_obstime=jwst_time2)

# Update acquisition target coordinates to scheduled JWST time
acqt_time = acqt.apply_space_motion(new_obstime=jwst_time)

# Print offset between original and updated coordinates
# of acquisition star (at JWST time)
offset_time = acqt.separation(acqt_time).to(u.mas)
pa_offset_time = acqt.position_angle(acqt_time).to(u.deg)
print('--> Offset between acquisition star (Gaia and JWST time):')
print(f'--> Total Offset:   {offset_time:.3f}')
print(f'--> Position angle: {pa_offset_time:.2f}')

--> Offset between acquisition star (Gaia and JWST time):
--> Total Offset:   43.346 mas
--> Position angle: 213.57 deg


## Calculate offset between acquisition star and R Aqr
The offset is calculated in radial offset from acquisition star to R Aqr, for the epochs of observation. The PA angle is calculated from EAST to NORTH. The calculations are done in the IRCS reference frame.

**Note that this is not the offset to use in APT because that requires an X & Y offset.** It is to be checked with STScI if the X & Y offset can be simply obtained by rotating the RA, DEC offset with the PA of the detector at the observation time, centered on the acquisition target. If this is the case then we need to know the accurate rotation angle of the instriment at observing time.

In [16]:
# Calculate offset for the beginning of the JWST observing window
offset1 = acqt_time1.separation(r_aqr2_time1).to(u.arcsec)
pa_offset1 = acqt_time1.position_angle(r_aqr2_time1).to(u.deg)
print('--> Offset between R Aqr 2 and acquisition star at window beginning.')
print("--> The (positive) position angle of the vector pointing from the")
print("--> acquisition star to R Aqr, measured East from North. ")
print(f'--> Total Offset:   {offset1:.3f}')
print(f'--> Position angle: {pa_offset1:.2f}')

--> Offset between R Aqr 2 and acquisition star at window beginning.
--> The (positive) position angle of the vector pointing from the
--> acquisition star to R Aqr, measured East from North. 
--> Total Offset:   52.001 arcsec
--> Position angle: 134.45 deg


In [17]:
# Calculate offset for the end of the JWST observing window
offset2 = acqt_time2.separation(r_aqr2_time2).to(u.arcsec)
pa_offset2 = acqt_time2.position_angle(r_aqr2_time2).to(u.deg)
print('--> Offset between R Aqr 2 and acquisition star at window end.')
print("--> The (positive) position angle of the vector pointing from the")
print("--> acquisition star to R Aqr, measured East from North. ")
print(f'--> Total Offset:   {offset2:.3f}')
print(f'--> Position angle: {pa_offset2:.2f}')

--> Offset between R Aqr 2 and acquisition star at window end.
--> The (positive) position angle of the vector pointing from the
--> acquisition star to R Aqr, measured East from North. 
--> Total Offset:   52.006 arcsec
--> Position angle: 134.45 deg


In [18]:
# Calculate offset for the nominal JWST observing time
offset = acqt_time.separation(r_aqr2_time).to(u.arcsec)
pa_offset = acqt_time.position_angle(r_aqr2_time).to(u.deg)
print('--> Offset between R Aqr 2 and acquisition star at JWST observing time.')
print("--> The (positive) position angle of the vector pointing from the")
print("--> acquisition star to R Aqr, measured East from North. ")
print(f'--> Total Offset:   {offset:.3f}')
print(f'--> Position angle: {pa_offset:.2f}')

--> Offset between R Aqr 2 and acquisition star at JWST observing time.
--> The (positive) position angle of the vector pointing from the
--> acquisition star to R Aqr, measured East from North. 
--> Total Offset:   52.002 arcsec
--> Position angle: 134.45 deg
